# Process GRACE Indicators

### Prepare Workspace

In [10]:
# Import system libraries
import os

# Import data manipulation libraries
import numpy as np
import pandas as pd

# Import geospatial libraries
from shapely.geometry import Polygon, mapping
import geopandas as gpd
import xarray as xr
import rioxarray

# Define the base directory and the subfolder names
base_dir = '/Users/jessicarapson/Documents/GitHub/water-supply-forecast' # Change this
nc_files_location = 'assets/data/grace_indicators'
gpkg_file_path = 'assets/data/geospatial.gpkg'

# Set the current working directory to the base directory
os.chdir(base_dir)

# Create the full path to the subfolders
nc_files_folder = os.path.join(base_dir, nc_files_location)

# Load GeoPackage file containing polygons
gdf = gpd.read_file(gpkg_file_path)

### Export Catchment Averages

In [29]:
# Load in site geospatial data
gdf_sites = gpd.read_file('assets/data/geospatial.gpkg')

# Initialize an empty list to store catchment bounding boxes
site_bboxes = []

# Iterate through each polygon (catchment) in the GeoDataFrame
for index, row in gdf_sites.iterrows():
    # Get the bounding box for each polygon
    bbox = row.geometry.bounds  # Extract the bounding box as (minx, miny, maxx, maxy)
    site_bboxes.append(bbox)  # Append the bounding box to the list

# Create an empty dataframe to store the derived data
dfs = []

# List all folders (years) in the subfolder
nc_files_folder = [f.path for f in os.scandir(nc_files_location) if f.is_dir()]

# Loop through each year's folder
for year_folder in nc_files_folder:
    
    # List all .nc4 files in the current year's folder
    nc_files = [f.path for f in os.scandir(year_folder) if f.is_file() and f.name.endswith('.nc4')]
    
    # Loop through each .nc4 file in the year's folder
    for nc_file in nc_files:
        
        # Open the NetCDF file using xarray
        data = xr.open_dataset(nc_file).rio.write_crs(gdf.crs)
        
        # Loop through the catchments
        for index, row in gdf.iterrows():
            
            # Select catchment area
            polygon = row['geometry']
            
            # Select only raster data that overlaps with the catchment
            data_clipped = data.rio.clip(gdf_sites.geometry.apply(mapping)[[index]], gdf_sites.crs)
            
            # Record the site and time
            site_id = gdf.iloc[index]['site_id']
            time =  pd.to_datetime(data_clipped.coords['time'].values.item())
            print("Processing GRACE for:", site_id, "at", time")

            # Take the cachment mean for each value
            gws_inst = np.nanmean(data_clipped['gws_inst'].values)
            rtzsm_inst = np.nanmean(data_clipped['rtzsm_inst'].values)
            sfsm_inst = np.nanmean(data_clipped['sfsm_inst'].values)
            
            # Create a dataframe with the selected data
            subset_df = pd.DataFrame({
                'site_id': site_id,
                'time': time,
                'mean_gws_inst': gws_inst,
                'mean_rtzsm_inst': rtzsm_inst,
                'mean_sfsm_inst': sfsm_inst}, index=[0])
        
            # Append the subset dataframe to the output dataframe
            dfs.append(subset_df)
        
        # Once done, close the dataset
        data.close()
        
# Concatenate all dataframes into a single dataframe
result_df = pd.DataFrame(pd.concat(dfs, ignore_index=True))

# Export the dataframe
result_df.to_csv("assets/data/grace_indicators/grace_aggregated.csv", index=False)

Processing GRACE for: hungry_horse_reservoir_inflow (1/26)
Processing GRACE for: snake_r_nr_heise (2/26)
Processing GRACE for: pueblo_reservoir_inflow (3/26)
Processing GRACE for: sweetwater_r_nr_alcova (4/26)
Processing GRACE for: missouri_r_at_toston (5/26)
Processing GRACE for: animas_r_at_durango (6/26)
Processing GRACE for: yampa_r_nr_maybell (7/26)
Processing GRACE for: libby_reservoir_inflow (8/26)
Processing GRACE for: boise_r_nr_boise (9/26)
Processing GRACE for: green_r_bl_howard_a_hanson_dam (10/26)
Processing GRACE for: taylor_park_reservoir_inflow (11/26)
Processing GRACE for: dillon_reservoir_inflow (12/26)
Processing GRACE for: ruedi_reservoir_inflow (13/26)
Processing GRACE for: fontenelle_reservoir_inflow (14/26)
Processing GRACE for: weber_r_nr_oakley (15/26)
Processing GRACE for: san_joaquin_river_millerton_reservoir (16/26)
Processing GRACE for: merced_river_yosemite_at_pohono_bridge (17/26)
Processing GRACE for: american_river_folsom_lake (18/26)
Processing GRACE f

Processing GRACE for: pecos_r_nr_pecos (25/26)
Processing GRACE for: owyhee_r_bl_owyhee_dam (26/26)
Processing GRACE for: hungry_horse_reservoir_inflow (1/26)
Processing GRACE for: snake_r_nr_heise (2/26)
Processing GRACE for: pueblo_reservoir_inflow (3/26)
Processing GRACE for: sweetwater_r_nr_alcova (4/26)
Processing GRACE for: missouri_r_at_toston (5/26)
Processing GRACE for: animas_r_at_durango (6/26)
Processing GRACE for: yampa_r_nr_maybell (7/26)
Processing GRACE for: libby_reservoir_inflow (8/26)
Processing GRACE for: boise_r_nr_boise (9/26)
Processing GRACE for: green_r_bl_howard_a_hanson_dam (10/26)
Processing GRACE for: taylor_park_reservoir_inflow (11/26)
Processing GRACE for: dillon_reservoir_inflow (12/26)
Processing GRACE for: ruedi_reservoir_inflow (13/26)
Processing GRACE for: fontenelle_reservoir_inflow (14/26)
Processing GRACE for: weber_r_nr_oakley (15/26)
Processing GRACE for: san_joaquin_river_millerton_reservoir (16/26)
Processing GRACE for: merced_river_yosemite_

Processing GRACE for: skagit_ross_reservoir (23/26)
Processing GRACE for: boysen_reservoir_inflow (24/26)
Processing GRACE for: pecos_r_nr_pecos (25/26)
Processing GRACE for: owyhee_r_bl_owyhee_dam (26/26)
Processing GRACE for: hungry_horse_reservoir_inflow (1/26)
Processing GRACE for: snake_r_nr_heise (2/26)
Processing GRACE for: pueblo_reservoir_inflow (3/26)
Processing GRACE for: sweetwater_r_nr_alcova (4/26)
Processing GRACE for: missouri_r_at_toston (5/26)
Processing GRACE for: animas_r_at_durango (6/26)
Processing GRACE for: yampa_r_nr_maybell (7/26)
Processing GRACE for: libby_reservoir_inflow (8/26)
Processing GRACE for: boise_r_nr_boise (9/26)
Processing GRACE for: green_r_bl_howard_a_hanson_dam (10/26)
Processing GRACE for: taylor_park_reservoir_inflow (11/26)
Processing GRACE for: dillon_reservoir_inflow (12/26)
Processing GRACE for: ruedi_reservoir_inflow (13/26)
Processing GRACE for: fontenelle_reservoir_inflow (14/26)
Processing GRACE for: weber_r_nr_oakley (15/26)
Proces

Processing GRACE for: detroit_lake_inflow (21/26)
Processing GRACE for: virgin_r_at_virtin (22/26)
Processing GRACE for: skagit_ross_reservoir (23/26)
Processing GRACE for: boysen_reservoir_inflow (24/26)
Processing GRACE for: pecos_r_nr_pecos (25/26)
Processing GRACE for: owyhee_r_bl_owyhee_dam (26/26)
Processing GRACE for: hungry_horse_reservoir_inflow (1/26)
Processing GRACE for: snake_r_nr_heise (2/26)
Processing GRACE for: pueblo_reservoir_inflow (3/26)
Processing GRACE for: sweetwater_r_nr_alcova (4/26)
Processing GRACE for: missouri_r_at_toston (5/26)
Processing GRACE for: animas_r_at_durango (6/26)
Processing GRACE for: yampa_r_nr_maybell (7/26)
Processing GRACE for: libby_reservoir_inflow (8/26)
Processing GRACE for: boise_r_nr_boise (9/26)
Processing GRACE for: green_r_bl_howard_a_hanson_dam (10/26)
Processing GRACE for: taylor_park_reservoir_inflow (11/26)
Processing GRACE for: dillon_reservoir_inflow (12/26)
Processing GRACE for: ruedi_reservoir_inflow (13/26)
Processing GR

Processing GRACE for: colville_r_at_kettle_falls (19/26)
Processing GRACE for: stehekin_r_at_stehekin (20/26)
Processing GRACE for: detroit_lake_inflow (21/26)
Processing GRACE for: virgin_r_at_virtin (22/26)
Processing GRACE for: skagit_ross_reservoir (23/26)
Processing GRACE for: boysen_reservoir_inflow (24/26)
Processing GRACE for: pecos_r_nr_pecos (25/26)
Processing GRACE for: owyhee_r_bl_owyhee_dam (26/26)
Processing GRACE for: hungry_horse_reservoir_inflow (1/26)
Processing GRACE for: snake_r_nr_heise (2/26)
Processing GRACE for: pueblo_reservoir_inflow (3/26)
Processing GRACE for: sweetwater_r_nr_alcova (4/26)
Processing GRACE for: missouri_r_at_toston (5/26)
Processing GRACE for: animas_r_at_durango (6/26)
Processing GRACE for: yampa_r_nr_maybell (7/26)
Processing GRACE for: libby_reservoir_inflow (8/26)
Processing GRACE for: boise_r_nr_boise (9/26)
Processing GRACE for: green_r_bl_howard_a_hanson_dam (10/26)
Processing GRACE for: taylor_park_reservoir_inflow (11/26)
Processing

...

Processing GRACE for: yampa_r_nr_maybell (7/26)
Processing GRACE for: libby_reservoir_inflow (8/26)
Processing GRACE for: boise_r_nr_boise (9/26)
Processing GRACE for: green_r_bl_howard_a_hanson_dam (10/26)
Processing GRACE for: taylor_park_reservoir_inflow (11/26)
Processing GRACE for: dillon_reservoir_inflow (12/26)
Processing GRACE for: ruedi_reservoir_inflow (13/26)
Processing GRACE for: fontenelle_reservoir_inflow (14/26)
Processing GRACE for: weber_r_nr_oakley (15/26)
Processing GRACE for: san_joaquin_river_millerton_reservoir (16/26)
Processing GRACE for: merced_river_yosemite_at_pohono_bridge (17/26)
Processing GRACE for: american_river_folsom_lake (18/26)
Processing GRACE for: colville_r_at_kettle_falls (19/26)
Processing GRACE for: stehekin_r_at_stehekin (20/26)
Processing GRACE for: detroit_lake_inflow (21/26)
Processing GRACE for: virgin_r_at_virtin (22/26)
Processing GRACE for: skagit_ross_reservoir (23/26)
Processing GRACE for: boysen_reservoir_inflow (24/26)
Processing G

Processing GRACE for: missouri_r_at_toston (5/26)
Processing GRACE for: animas_r_at_durango (6/26)
Processing GRACE for: yampa_r_nr_maybell (7/26)
Processing GRACE for: libby_reservoir_inflow (8/26)
Processing GRACE for: boise_r_nr_boise (9/26)
Processing GRACE for: green_r_bl_howard_a_hanson_dam (10/26)
Processing GRACE for: taylor_park_reservoir_inflow (11/26)
Processing GRACE for: dillon_reservoir_inflow (12/26)
Processing GRACE for: ruedi_reservoir_inflow (13/26)
Processing GRACE for: fontenelle_reservoir_inflow (14/26)
Processing GRACE for: weber_r_nr_oakley (15/26)
Processing GRACE for: san_joaquin_river_millerton_reservoir (16/26)
Processing GRACE for: merced_river_yosemite_at_pohono_bridge (17/26)
Processing GRACE for: american_river_folsom_lake (18/26)
Processing GRACE for: colville_r_at_kettle_falls (19/26)
Processing GRACE for: stehekin_r_at_stehekin (20/26)
Processing GRACE for: detroit_lake_inflow (21/26)
Processing GRACE for: virgin_r_at_virtin (22/26)
Processing GRACE fo

Processing GRACE for: pueblo_reservoir_inflow (3/26)
Processing GRACE for: sweetwater_r_nr_alcova (4/26)
Processing GRACE for: missouri_r_at_toston (5/26)
Processing GRACE for: animas_r_at_durango (6/26)
Processing GRACE for: yampa_r_nr_maybell (7/26)
Processing GRACE for: libby_reservoir_inflow (8/26)
Processing GRACE for: boise_r_nr_boise (9/26)
Processing GRACE for: green_r_bl_howard_a_hanson_dam (10/26)
Processing GRACE for: taylor_park_reservoir_inflow (11/26)
Processing GRACE for: dillon_reservoir_inflow (12/26)
Processing GRACE for: ruedi_reservoir_inflow (13/26)
Processing GRACE for: fontenelle_reservoir_inflow (14/26)
Processing GRACE for: weber_r_nr_oakley (15/26)
Processing GRACE for: san_joaquin_river_millerton_reservoir (16/26)
Processing GRACE for: merced_river_yosemite_at_pohono_bridge (17/26)
Processing GRACE for: american_river_folsom_lake (18/26)
Processing GRACE for: colville_r_at_kettle_falls (19/26)
Processing GRACE for: stehekin_r_at_stehekin (20/26)
Processing GR

Processing GRACE for: hungry_horse_reservoir_inflow (1/26)
Processing GRACE for: snake_r_nr_heise (2/26)
Processing GRACE for: pueblo_reservoir_inflow (3/26)
Processing GRACE for: sweetwater_r_nr_alcova (4/26)
Processing GRACE for: missouri_r_at_toston (5/26)
Processing GRACE for: animas_r_at_durango (6/26)
Processing GRACE for: yampa_r_nr_maybell (7/26)
Processing GRACE for: libby_reservoir_inflow (8/26)
Processing GRACE for: boise_r_nr_boise (9/26)
Processing GRACE for: green_r_bl_howard_a_hanson_dam (10/26)
Processing GRACE for: taylor_park_reservoir_inflow (11/26)
Processing GRACE for: dillon_reservoir_inflow (12/26)
Processing GRACE for: ruedi_reservoir_inflow (13/26)
Processing GRACE for: fontenelle_reservoir_inflow (14/26)
Processing GRACE for: weber_r_nr_oakley (15/26)
Processing GRACE for: san_joaquin_river_millerton_reservoir (16/26)
Processing GRACE for: merced_river_yosemite_at_pohono_bridge (17/26)
Processing GRACE for: american_river_folsom_lake (18/26)
Processing GRACE f

Processing GRACE for: pecos_r_nr_pecos (25/26)
Processing GRACE for: owyhee_r_bl_owyhee_dam (26/26)
